In [1]:
# Importing necessary libraries
import os
import random
import shutil
import numpy as np
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import VGG16, VGG19, ResNet50, InceptionV3, Xception
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
from sklearn.metrics import classification_report, confusion_matrix

# Setting random seed for reproducibility
np.random.seed(42)
random.seed(42)

# Define constants
NUM_CLASSES = 2
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
TRAIN_DIR = 'train'
VAL_DIR = 'val'
TEST_DIR = 'test'
TRAIN_SPLIT = 0.7
VAL_SPLIT = 0.1
TEST_SPLIT = 0.2
test_samples = 280

In [2]:
# Define the base directory where the image data is located
BASE_DIR = r"C:\Users\mohit\Downloads\DIP Assignment 2\Data"

In [3]:
# Set the paths for training, validation, and test data
TRAIN_DIR = os.path.join(BASE_DIR, 'train')
VAL_DIR = os.path.join(BASE_DIR, 'val')
TEST_DIR = os.path.join(BASE_DIR, 'test')

In [4]:
# Load and preprocess the data
train_datagen = ImageDataGenerator(rescale=1./255,shear_range=0.2,zoom_range=0.2,horizontal_flip=True)
val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

In [5]:
train_generator = train_datagen.flow_from_directory(TRAIN_DIR,target_size=IMG_SIZE,batch_size=BATCH_SIZE,class_mode='categorical')

val_generator = val_datagen.flow_from_directory(VAL_DIR,target_size=IMG_SIZE,batch_size=BATCH_SIZE,class_mode='categorical')

test_generator = test_datagen.flow_from_directory(TEST_DIR,target_size=IMG_SIZE,batch_size=BATCH_SIZE,class_mode='categorical',shuffle=False)

Found 980 images belonging to 2 classes.
Found 140 images belonging to 2 classes.
Found 280 images belonging to 2 classes.


In [6]:
# Define function for creating transfer learning models
def create_transfer_model(base_model, num_classes):
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation='relu')(x)
    predictions = Dense(num_classes, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=predictions)
    return model

# Define VGG16 model
vgg16_base = VGG16(include_top=False, weights='imagenet', input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3))
vgg16_model = create_transfer_model(vgg16_base, NUM_CLASSES)

# Define VGG19 model
vgg19_base = VGG19(include_top=False, weights='imagenet', input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3))
vgg19_model = create_transfer_model(vgg19_base, NUM_CLASSES)

In [7]:
# Compile the models
optimizer = Adam(learning_rate=0.0001)
vgg16_model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
vgg19_model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [8]:
# Define checkpoint callback to save the best model during training
checkpoint = ModelCheckpoint('best_model.h5', monitor='val_loss', save_best_only=True)

In [9]:
# Train the models
vgg16_history = vgg16_model.fit_generator(train_generator, steps_per_epoch=train_generator.n // train_generator.batch_size, epochs=5, validation_data=val_generator, validation_steps=val_generator.n // val_generator.batch_size, callbacks=[checkpoint])
vgg19_history = vgg19_model.fit_generator(train_generator, steps_per_epoch=train_generator.n // train_generator.batch_size, epochs=5, validation_data=val_generator, validation_steps=val_generator.n // val_generator.batch_size, callbacks=[checkpoint])

C:\Users\mohit\AppData\Local\Temp\ipykernel_25572\2039408895.py:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  vgg16_history = vgg16_model.fit_generator(train_generator, steps_per_epoch=train_generator.n // train_generator.batch_size, epochs=5, validation_data=val_generator, validation_steps=val_generator.n // val_generator.batch_size, callbacks=[checkpoint])


Epoch 1/5
30/30 [==============================] - 269s 9s/step - loss: 0.6998 - accuracy: 0.5190 - val_loss: 0.7125 - val_accuracy: 0.5000
Epoch 2/5
30/30 [==============================] - 286s 10s/step - loss: 0.6402 - accuracy: 0.6118 - val_loss: 0.4709 - val_accuracy: 0.8281
Epoch 3/5
30/30 [==============================] - 470s 16s/step - loss: 0.2767 - accuracy: 0.9008 - val_loss: 0.1435 - val_accuracy: 0.9531
Epoch 4/5
30/30 [==============================] - 503s 17s/step - loss: 0.1493 - accuracy: 0.9473 - val_loss: 0.1466 - val_accuracy: 0.9609
Epoch 5/5
30/30 [==============================] - 504s 17s/step - loss: 0.1644 - accuracy: 0.9494 - val_loss: 0.0989 - val_accuracy: 0.9688


C:\Users\mohit\AppData\Local\Temp\ipykernel_25572\2039408895.py:3: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  vgg19_history = vgg19_model.fit_generator(train_generator, steps_per_epoch=train_generator.n // train_generator.batch_size, epochs=5, validation_data=val_generator, validation_steps=val_generator.n // val_generator.batch_size, callbacks=[checkpoint])


Epoch 1/5
30/30 [==============================] - 570s 19s/step - loss: 0.7282 - accuracy: 0.5042 - val_loss: 0.6897 - val_accuracy: 0.5156
Epoch 2/5
30/30 [==============================] - 558s 19s/step - loss: 0.6453 - accuracy: 0.6983 - val_loss: 0.4191 - val_accuracy: 0.8672
Epoch 3/5
30/30 [==============================] - 544s 18s/step - loss: 0.3698 - accuracy: 0.8576 - val_loss: 0.1742 - val_accuracy: 0.9297
Epoch 4/5
30/30 [==============================] - 547s 18s/step - loss: 0.2672 - accuracy: 0.9040 - val_loss: 0.1718 - val_accuracy: 0.9375
Epoch 5/5
30/30 [==============================] - 354s 12s/step - loss: 0.2497 - accuracy: 0.9146 - val_loss: 0.2170 - val_accuracy: 0.8984


In [10]:
# Evaluate the models on test data
vgg16_scores = vgg16_model.evaluate_generator(test_generator, steps=test_samples // BATCH_SIZE)
print("VGG16 Test Loss:", vgg16_scores[0])
print("VGG16 Test Accuracy:", vgg16_scores[1])

vgg19_scores = vgg19_model.evaluate_generator(test_generator, steps=test_samples // BATCH_SIZE)
print("VGG19 Test Loss:", vgg19_scores[0])
print("VGG19 Test Accuracy:", vgg19_scores[1])

C:\Users\mohit\AppData\Local\Temp\ipykernel_25572\1392354301.py:2: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  vgg16_scores = vgg16_model.evaluate_generator(test_generator, steps=test_samples // BATCH_SIZE)


VGG16 Test Loss: 0.07796119153499603
VGG16 Test Accuracy: 0.97265625


C:\Users\mohit\AppData\Local\Temp\ipykernel_25572\1392354301.py:6: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  vgg19_scores = vgg19_model.evaluate_generator(test_generator, steps=test_samples // BATCH_SIZE)


VGG19 Test Loss: 0.14393511414527893
VGG19 Test Accuracy: 0.94140625


In [15]:
# Get the number of test samples
num_test_samples = test_generator.n

# Calculate the number of steps for prediction
num_prediction_steps = num_test_samples // test_generator.batch_size + 1

# Generate predictions for all test samples
vgg16_predictions = vgg16_model.predict_generator(test_generator, steps=num_prediction_steps, verbose=1)

# Convert predictions to class labels
vgg16_predicted_labels = np.argmax(vgg16_predictions, axis=1)

C:\Users\mohit\AppData\Local\Temp\ipykernel_25572\2218791254.py:8: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  vgg16_predictions = vgg16_model.predict_generator(test_generator, steps=num_prediction_steps, verbose=1)


9/9 [==============================] - 23s 3s/step


In [16]:
# Get the number of test samples
num_test_samples = test_generator.n

# Calculate the number of steps for prediction
num_prediction_steps = num_test_samples // test_generator.batch_size + 1

# Generate predictions for all test samples
vgg19_predictions = vgg19_model.predict_generator(test_generator, steps=num_prediction_steps, verbose=1)

# Convert predictions to class labels
vgg19_predicted_labels = np.argmax(vgg19_predictions, axis=1)

C:\Users\mohit\AppData\Local\Temp\ipykernel_25572\903632277.py:8: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  vgg19_predictions = vgg19_model.predict_generator(test_generator, steps=num_prediction_steps, verbose=1)


9/9 [==============================] - 30s 3s/step


In [17]:
# Get true class labels
true_labels = test_generator.classes

In [18]:
# Calculate classification report
vgg16_report = classification_report(true_labels, vgg16_predicted_labels)

In [19]:
# Calculate classification report
from sklearn.metrics import classification_report

# Get the ground truth labels
ground_truth_labels = test_generator.classes

# Get the predicted labels
vgg19_predicted_labels = np.argmax(vgg19_predictions, axis=1)

# Calculate classification report
classification_report_vgg19 = classification_report(ground_truth_labels, vgg19_predicted_labels, zero_division=1)

In [20]:
print("vgg16 Classification Report:")
print(vgg16_report)

print("vgg19 Classification Report:")
print(classification_report_vgg19)

vgg16 Classification Report:
              precision    recall  f1-score   support

           0       0.95      0.99      0.97       140
           1       0.99      0.94      0.96       140

    accuracy                           0.96       280
   macro avg       0.97      0.96      0.96       280
weighted avg       0.97      0.96      0.96       280

vgg19 Classification Report:
              precision    recall  f1-score   support

           0       0.87      0.99      0.93       140
           1       0.98      0.86      0.92       140

    accuracy                           0.92       280
   macro avg       0.93      0.92      0.92       280
weighted avg       0.93      0.92      0.92       280



In [ ]:
import matplotlib.pyplot as plt

# Train the VGG16 model and obtain the training history
vgg16_history = vgg16_model.fit(train_generator, steps_per_epoch=train_generator.n // train_generator.batch_size, epochs=5)

# Plot the training loss curve
plt.plot(vgg16_history.history['loss'])
plt.title('vgg16 Training Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.show()

Epoch 1/5
30/30 [==============================] - 608s 21s/step - loss: 0.1287 - accuracy: 0.9599
Epoch 2/5
30/30 [==============================] - 802s 27s/step - loss: 0.1328 - accuracy: 0.9620
Epoch 3/5
15/30 [==============>...............] - ETA: 6:36 - loss: 0.0728 - accuracy: 0.9854

In [ ]:
import matplotlib.pyplot as plt

# Train the VGG16 model and obtain the training history
vgg19_history = vgg19_model.fit(train_generator, steps_per_epoch=train_generator.n // train_generator.batch_size, epochs=10)

# Plot the training loss curve
plt.plot(vgg19_history.history['loss'])
plt.title('vgg19 Training Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.show()

In [ ]:
plt.plot(vgg16_history.history['accuracy'])
plt.title('VGG16 Training Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.show()

In [ ]:
plt.plot(vgg19_history.history['accuracy'])
plt.title('VGG19 Training Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.show()